# Análise de perda de carga ao longo de uma instalação de recalque

## Sistema de Recalque

O sistema de recalque ou de adução é um sistema posicionado após uma bomba sucção que bombeia o fluido de um lugar inferior para um superior. É muito utilizado em residencias, prédios, industria e setor agrícola.

### Exemplo de uma instalação de recalque
![Alt text](./img/sistema_de_recalque.png)

Fonte: João Carlos Barros Almeida da Silva

### Perdas localizadas e distribuidas

In [88]:
from math import pow, pi, sqrt
import pandas as pd

# Constantes
g = 9.81
D = 0.090
mi = 0.7978*pow(10,-3)
ro = pow(10,3)
e = 0.009*pow(10,-3)

# Calculo da velocidade
v = (2.46 * 4) / (pi * pow(D, 2))

# Data for the Series objects
data = {
    'Nome': ['Curva de 90°', 'Válvula de retenção', 'Curva de 45°', 'Registro de gaveta', 'Válvula de pé'],
    'Quantidade': [5, 2, 9, 3, 1],
    'Coeficiente de perda localizada (K)': [0.4, 2.5, 0.2, 0.2, 1.75],
    'Perda localizada (h)': []
}

# Calculate the 'Perda localizada (h)' values
for i in range(len(data['Nome'])):
    perda_localizada = (data['Quantidade'][i] * data['Coeficiente de perda localizada (K)'][i] * pow(v, 2)) / (2 * g)
    data['Perda localizada (h)'].append(perda_localizada)

# Create the DataFrame
df = pd.DataFrame(data)

# Format the last column with two decimal places
df['Perda localizada (h)'] = df['Perda localizada (h)'].map(lambda x: round(x, 2))

# Output the DataFrame
df


,Nome,Quantidade,Coeficiente de perda localizada (K),Perda localizada (h)
0,Curva de 90°,5,0.40,15242.33
1,Válvula de retenção,2,2.50,38105.83
2,Curva de 45°,9,0.20,13718.10
3,Registro de gaveta,3,0.20,4572.70
4,Válvula de pé,1,1.75,13337.04


In [95]:
erro = 0.000001

In [97]:
re = (ro*v*D) / mi
print(f"Número de Reynolds = {re:.2e}")


Número de Reynolds = 43622312.4710802


In [99]:
def colebrook_white(e, D, Re):
    f = 0.25 * pow(log((e / (D * 3.7)) + (5.74 / pow(re, 0.9)), 10), -2)  # Valor inicial para f
    delta = 1  # Valor inicial para a diferença entre f atual e f anterior

    while delta > erro:  # Precisão desejada para o resultado
        x = -2*log((e / (3.7 * D)) + (2.51 / (Re * sqrt(f))),10)
        f_new = 1 / (x ** 2)  # Novo valor de f

        delta = abs(f - f_new)  # Calcula a diferença entre f atual e f anterior
        f = f_new  # Atualiza o valor de f

    return f
print(colebrook_white(e, D, re))

0.012023718505045786
